<a href="https://colab.research.google.com/github/mavillot/FUNSD-Entity-Linking/blob/main/RoBerta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de lenguaje

## Data preparation

### Dataset:

In [1]:
%%capture
!wget https://guillaumejaume.github.io/FUNSD/dataset.zip -O dataset.zip
!unzip dataset.zip

### Extracting candidates 

In [2]:
import json
from pathlib import Path
import cv2
import numpy as np

In [3]:
def candidates(path):
  anot=json.loads(open(path).read())
  img=Path(path).stem+'.png'
  path_image=Path(path).parent.parent/'images'/img
  h,w,_=cv2.imread(str(path_image)).shape
  question, answer=[],[]
  text={}
  for block in anot['form']:
    if block['label']=='question':
      question.append([block['id'],block['box']])
      text[block['id']]=block['text']
    if block['label']=='answer':
      answer.append([block['id'],block['box']])
      text[block['id']]=block['text']
  # We have built 2 list with answers and questions' info 
  #For each answer we look for its question
  dic={}
  for a in answer:
    bbox=a[1]
    candidates=[]
    candidates_more=[]
    x_a=int(bbox[0])
    x1_a=int(bbox[2])
    y_a=int(bbox[1])
    y1_a=int(bbox[3])
    pto_a=[(x_a+x1_a)/2,(y_a+y1_a)/2]
    for q in question:
      bbox=q[1]
      x_q=int(bbox[0])
      x1_q=int(bbox[2])
      y_q=int(bbox[1])
      y1_q=int(bbox[3])
      pto_q=[(x_q+x1_q)/2,(y_q+y1_q)/2]
      if x_q<x1_a+0.05*w and y1_q>y_a-0.1*h and y_q<y1_a+0.01*h:
        dist=np.sqrt((x_a-x1_q)**2+(pto_a[1]-pto_q[1])**2)
        candidates.append([q[0],dist])
      if x_q<x1_a +0.05*w and y1_q>y_a-0.6*h and y_q<y1_a+0.03*h:
        dist=np.sqrt((x_a-x1_q)**2+(pto_a[1]-pto_q[1])**2)
        candidates_more.append([q[0],dist])
    if candidates!=[]:
      dic[a[0]]=candidates
    else:
      dic[a[0]]=candidates_more
  return dic,text

In [4]:
candidates('dataset/training_data/annotations/0000971160.json')

({4: [[1, 286.05287972680856],
   [3, 41.593268686170845],
   [10, 309.0457085934053],
   [13, 20.006249023742555],
   [15, 293.0959740426334]],
  11: [[3, 303.0264014900352], [10, 11.10180165558726]],
  12: [[3, 25.243811122728676], [10, 339.1194479825656]],
  14: [[1, 11.10180165558726],
   [10, 50.835519078691426],
   [15, 18.33712082089225]],
  16: [[1, 82.76472678623425],
   [2, 39.96248240537617],
   [3, 349.7802881810237],
   [10, 116.84177335182824],
   [13, 311.4498354470588],
   [15, 78.18567643756751]],
  18: [[17, 26.419689627245813]],
  21: [[5, 15.402921800749363],
   [6, 254.32656172724074],
   [7, 333.7813655673426],
   [20, 21.587033144922902]],
  22: [[5, 118.71394189394942], [20, 112.58885380000989]]},
 {1: ':',
  2: 'Suggestion:',
  3: 'Date:',
  4: 'Licensee',
  5: '',
  6: 'Yes',
  7: 'No',
  10: 'Name / Phone Ext. :',
  11: 'M. Hamann P. Harper, P. Martinez',
  12: '9/ 3/ 92',
  13: 'R&D Group:',
  14: 'J. S. Wigand',
  15: 'Supervisor / Manager',
  16: 'Disconti

We rank the candidates according to the distance

In [5]:
from operator import itemgetter

In [6]:
def rank(path):
  cand,text=candidates(path)
  for c in cand:
    aux=cand[c]
    cand[c]=[l[0] for l in sorted(aux, key=itemgetter(1))]
  return cand,text

In [7]:
rank('dataset/training_data/annotations/0000971160.json')

({4: [13, 3, 1, 15, 10],
  11: [10, 3],
  12: [3, 10],
  14: [1, 15, 10],
  16: [2, 15, 1, 10, 13, 3],
  18: [17],
  21: [5, 20, 6, 7],
  22: [20, 5]},
 {1: ':',
  2: 'Suggestion:',
  3: 'Date:',
  4: 'Licensee',
  5: '',
  6: 'Yes',
  7: 'No',
  10: 'Name / Phone Ext. :',
  11: 'M. Hamann P. Harper, P. Martinez',
  12: '9/ 3/ 92',
  13: 'R&D Group:',
  14: 'J. S. Wigand',
  15: 'Supervisor / Manager',
  16: 'Discontinue coal retention analyses on licensee submitted product samples (Note : Coal Retention testing is not performed by most licensees. Other B&W physical measurements as ends stability and inspection for soft spots in ciparettes are thought to be sufficient measures to assure cigarette physical integrity. The proposed action will increase laboratory productivity . )',
  17: 'Suggested Solutions (s) :',
  18: 'Delete coal retention from the list of standard analyses performed on licensee submitted product samples. Special requests for coal retention testing could still be sub

We construct now a function that given a path it returns for each answer the candidates, the labels and the text of the questions and answers

In [8]:
def candidatesANDlabels(path):
  anot=json.loads(open(path).read())
  cand,text=rank(path)
  dic_label={}
  for block in anot['form']:
    if block['label']=='answer':
      id=block['id']
      id_question=[]
      for link in block['linking']:
        if link[0]==id:
          id_question.append(link[1])
        else: id_question.append(link[0])
      lista=cand[id]
      labels=[lista[i] in id_question for i in range(0,len(lista))]
      dic_label[id]=labels
  return cand,dic_label,text     

In [9]:
candidatesANDlabels('dataset/training_data/annotations/0000971160.json')

({4: [13, 3, 1, 15, 10],
  11: [10, 3],
  12: [3, 10],
  14: [1, 15, 10],
  16: [2, 15, 1, 10, 13, 3],
  18: [17],
  21: [5, 20, 6, 7],
  22: [20, 5]},
 {4: [True, False, False, False, False],
  11: [True, False],
  12: [True, False],
  14: [False, True, False],
  16: [True, False, False, False, False, False],
  18: [True],
  21: [False, True, False, False],
  22: [True, False]},
 {1: ':',
  2: 'Suggestion:',
  3: 'Date:',
  4: 'Licensee',
  5: '',
  6: 'Yes',
  7: 'No',
  10: 'Name / Phone Ext. :',
  11: 'M. Hamann P. Harper, P. Martinez',
  12: '9/ 3/ 92',
  13: 'R&D Group:',
  14: 'J. S. Wigand',
  15: 'Supervisor / Manager',
  16: 'Discontinue coal retention analyses on licensee submitted product samples (Note : Coal Retention testing is not performed by most licensees. Other B&W physical measurements as ends stability and inspection for soft spots in ciparettes are thought to be sufficient measures to assure cigarette physical integrity. The proposed action will increase laborator

### Dataset Class

We create this class in order to obtain the dataset we will use for training the BERT model. 
For each form we extract all the answers and its candidates for question. For each candidate we concatenate its text with the answer text. If the link between those 2 entities exists the label will be: 1 and if it doesn't exist it will be 0.

In [10]:
import os
class Dataset():
    def __init__(self, path_annotation):
        self.path_annotation= path_annotation

    def __iter__(self):
      with os.scandir(self.path_annotation) as files:
        for file in files:
          yield file.name
          
    def __len__(self):
      i=0
      for file in self:
        i+=1
      return i

    def textList(self, o):
      path=self.path_annotation+'/'+o
      cand,label,text=candidatesANDlabels(path)
      question_answer,et=[],[]
      dic={True:1,False:0}
      for c in cand:
        question_answer=question_answer+[text[x]+' '+text[c] for x in cand[c]]
        et=et+[dic[z] for z in label[c]]
      return question_answer,et
    
    def preparation(self):
      text=[]
      label=[]
      for file in self:
        txt,lbl=self.textList(file)
        text=text+txt
        label=label+lbl
      return (text,label)

In [11]:
dataset_train=Dataset('dataset/training_data/annotations')
dataset_test=Dataset('dataset/testing_data/annotations')

In [12]:
text_train, labels_train=dataset_train.preparation()
text_test, labels_test=dataset_test.preparation()

In [13]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(text_train, labels_train, test_size=.2)

**We check if all the answers have at least one question in their candidates:**

In [14]:
path='/content/dataset/training_data/annotations/'
i=0
for f in dataset_train:
  f=path+f
  cand,_,_=candidatesANDlabels(f)
  for c in cand:
    if cand[c]==[]:
      i+=1
print(i)

0


There's no answer without candidates

**We check if all the correct questions are in the candidate list of the answer:**

In [15]:
path='/content/dataset/training_data/annotations/'
i,j=0,0
for f in dataset_train:
  f=path+f
  cand,lbl,text=candidatesANDlabels(f)
  for k in lbl:
    j+=1
    if not(True in lbl[k]):
      i+=1
print(i,j)

47 2802


In [16]:
47*100/2802

1.6773733047822983

Ther're 47 of the 2802 answers (1.68%) who haven't got their correct question in the candidates

### FUNSDataset Class

This class helps us to pass the dataset in a format that the model understands.

In [17]:
import torch
class FUNDSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## Model

In [18]:
%%capture
pip install transformers

#### Metrics

These are the metrics we will be using for training.

In [19]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

##### mAP

In [20]:
import numpy as np
from sklearn.metrics import average_precision_score

def mAP_x(scores, labels):
  m=average_precision_score(labels, scores)
  if np.isnan(m):
    return 0
  else:
    return m

In [21]:
def mAP(dataset,coef_FINAL,labels):
  i=0
  map=[]
  for f in dataset:
    cand,txt=rank(dataset.path_annotation+ '/' +f)
    n=len(cand)
    if n>0:
      for c in cand:
        j=i+len(cand[c])
        map.append(mAP_x(coef_FINAL[i:j],labels[i:j]))
        i=j
  return sum(map)/len(map)

##### mRank

In [22]:
import numpy as np
def mRank_x(predictions, labels,coef):
  if predictions==labels:
    return 0
  else:
    if 1 in labels:
      k=labels.index(1)
      return sum([c>coef[k] for c in coef])
    else:
      return len(predictions)

In [23]:
def mRank(dataset,prediction,labels,coef):
  mrank=[]
  i=0
  for f in dataset:
    cand,txt=rank(dataset.path_annotation+ '/' +f)
    n=len(cand)
    if n>0: #comprobamos que el documento tenga algun par clave-valor
      for c in cand:
        j=i+len(cand[c])
        if j!=i:
          mrank.append(mRank_x(prediction[i:j],labels[i:j],coef[i:j]))
          i=j
  return sum(mrank)/len(mrank)

#### Tokenizer and the model

In [ ]:
from transformers import BertForSequenceClassification,AutoTokenizer,Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = BertForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

You are using a model of type xlm-roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.1.intermediate.dense.weight', 'roberta.encoder.layer.2.intermediate.dense.bias', 'roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.pooler.dense.bias', 'roberta.encoder.layer.1.attention.self.key.bias', 'roberta.encoder.layer.5.intermediate.dense.bias', 'lm_head.dense.bias', 'roberta.encoder.layer.1.attention.self.key.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.4.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.9.attention.self.query.bias', 'roberta.encoder.layer.10.output.dense.weight', 'roberta.encoder.layer.7.output.LayerNorm.bias', 'roberta.encoder.layer.6.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.dens

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(text_test, truncation=True, padding=True)

In [ ]:
train_dataset = FUNDSDataset(train_encodings, train_labels)
val_dataset = FUNDSDataset(val_encodings, val_labels)
test_dataset = FUNDSDataset(test_encodings, labels_test)

#### Training arguments:

In [ ]:
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Entrenamiento

In [ ]:
trainer.train() 

***** Running training *****
  Num examples = 10502
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3942
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.540900,0.536175,0.786748,0.000000,0.000000,0.000000
2,0.542600,0.528542,0.789414,0.024691,1.000000,0.012500
3,0.513300,0.508967,0.791318,0.151703,0.569767,0.087500


***** Running Evaluation *****
  Num examples = 2626
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results/checkpoint-657
Configuration saved in ./results/checkpoint-657/config.json
Model weights saved in ./results/checkpoint-657/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-657/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-657/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
****

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.540900,0.536175,0.786748,0.000000,0.000000,0.000000
2,0.542600,0.528542,0.789414,0.024691,1.000000,0.012500
3,0.513300,0.508967,0.791318,0.151703,0.569767,0.087500
4,0.477400,0.530212,0.790937,0.148837,0.564706,0.085714
5,0.458000,0.558091,0.770373,0.221935,0.400000,0.153571
6,0.450500,0.551587,0.768469,0.220513,0.390909,0.153571


***** Running Evaluation *****
  Num examples = 2626
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2628
Configuration saved in ./results/checkpoint-2628/config.json
Model weights saved in ./results/checkpoint-2628/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2628/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2628/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
***** Running Evaluation *****
  Num examples = 2626
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-3285
Configuration saved in ./results/checkpoint-3285/config.json
Model weights saved in ./results/checkpoint-3285/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint

TrainOutput(global_step=3942, training_loss=0.4909772016504947, metrics={'train_runtime': 4376.2995, 'train_samples_per_second': 14.398, 'train_steps_per_second': 0.901, 'total_flos': 5148604409052240.0, 'train_loss': 0.4909772016504947, 'epoch': 6.0})

Saving the model

In [ ]:
trainer.save_model('RoBerta')

Saving model checkpoint to RoBerta
Configuration saved in RoBerta/config.json
Model weights saved in RoBerta/pytorch_model.bin
tokenizer config file saved in RoBerta/tokenizer_config.json
Special tokens file saved in RoBerta/special_tokens_map.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.make_archive('/content/drive/MyDrive/RoBerta', 'zip', 'RoBerta')

'/content/drive/MyDrive/RoBerta.zip'

## Evaluation

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 2110
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


{'epoch': 3.0,
 'eval_accuracy': 0.6341232227488152,
 'eval_f1': 0.38632750397456284,
 'eval_loss': 0.6881266236305237,
 'eval_precision': 0.43548387096774194,
 'eval_recall': 0.34714285714285714,
 'eval_runtime': 51.9692,
 'eval_samples_per_second': 40.601,
 'eval_steps_per_second': 5.08}

## Downloading and loading the trained weights

In [25]:
import shutil
!wget https://www.dropbox.com/s/639xxc3o7v9ena8/RoBerta.zip
!unzip RoBerta.zip

--2021-10-15 07:13:15--  https://www.dropbox.com/s/639xxc3o7v9ena8/RoBerta.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/639xxc3o7v9ena8/RoBerta.zip [following]
--2021-10-15 07:13:15--  https://www.dropbox.com/s/raw/639xxc3o7v9ena8/RoBerta.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc14d15ddfc4798d63d35eabfba.dl.dropboxusercontent.com/cd/0/inline/BYGPko5VSprSX_fZrm7WYL9lbM3dQp7GaKGxrT4C-NlTo6d5f1kKTJYKs5tGELFd1KMlnJsw8dmEtVqW1NxOvAGUFtP_fOz43hw3WfM8lwKe_rjI6V21GtpIKQ5cghB0yOVwmTG8fCjPO_kOw9w9X11E/file# [following]
--2021-10-15 07:13:15--  https://ucc14d15ddfc4798d63d35eabfba.dl.dropboxusercontent.com/cd/0/inline/BYGPko5VSprSX_fZrm7WYL9lbM3dQp7GaKGxrT4C-NlTo6d5f1kKTJYKs5tGELFd1KMlnJsw8dmEtVqW1NxOvAGUFtP_f

In [26]:
from transformers import BertForSequenceClassification,AutoTokenizer,Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained('RoBerta')
model = BertForSequenceClassification.from_pretrained('RoBerta', num_labels=2)
trainer = Trainer(model,compute_metrics=compute_metrics)

# Results

We build our test dataset

In [27]:
dataset_test=Dataset('dataset/testing_data/annotations')
text_test, labels_test=dataset_test.preparation()
test_encodings = tokenizer(text_test, truncation=True, padding=True)
test_dataset = FUNDSDataset(test_encodings, labels_test)

**We check if all the answers have at least one question in their candidates:**

In [28]:
path='/content/dataset/testing_data/annotations/'
i=0
for f in dataset_test:
  f=path+f
  cand,_,_=candidatesANDlabels(f)
  for c in cand:
    if cand[c]==[]:
      i+=1
print(i)

0


There's no answer without candidates

**We check if all the correct questions are in the candidate list of the answer:**

In [29]:
path='/content/dataset/testing_data/annotations/'
i,j=0,0
for f in dataset_test:
  f=path+f
  _,lbl,_=candidatesANDlabels(f)
  for k in lbl:
    j+=1
    if not(True in lbl[k]):
      i+=1
print(i,j)

3 821


In [30]:
3*100/821

0.3654080389768575

Ther're 3 of the 821 answers (0.37%) who haven't got their correct question in the candidates

### PREDICTIONS

In [31]:
coef=trainer.predict(test_dataset)[0]
prediction=coef.argmax(-1)
prediction=[p for p in prediction]
coef=[tupla[1] for tupla in coef]

***** Running Prediction *****
  Num examples = 4002
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Results (Before the rules are applied)

In [32]:
precision, recall, f1, _ = precision_recall_fscore_support(labels_test, prediction, average='binary')
acc = accuracy_score(labels_test, prediction)
print('accuracy: ',acc)
print('precision: ',precision)
print('recall: ',recall)
print('f1: ',f1)

accuracy:  0.7626186906546727
precision:  0.34366925064599485
recall:  0.16043425814234016
f1:  0.21875


In [33]:
mAP(dataset_test,coef,labels_test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


0.5822398774591209

In [34]:
mRank(dataset_test,prediction,labels_test,coef)

1.5529841656516443

### Results (After the rules are applied)

In [37]:
def rules(cand, pred,coef):
  i=0
  pred_FINAL,coef_FINAL=[],[]
  for c in cand:
    j=i+len(cand[c])
    new_pred=pred[i:j]
    new_coef=coef[i:j]
    if new_pred!=[]:
      if 1 in new_pred:
        encontrado=False
        m=0
        while encontrado==False:
          if new_pred[m]==1:
            encontrado=True
            new_coef[m]=10
          m+=1
        for k in range(m,j-i):
          new_pred[k]=0
      else:
        new_pred[0]=1
        new_coef[0]=10
      i=j
      pred_FINAL=pred_FINAL+new_pred
      coef_FINAL=coef_FINAL+new_coef
  return pred_FINAL,coef_FINAL

RULES:

In [38]:
path=dataset_test.path_annotation+'/'
a=0
prediction_FINAL,coef_FINAL=[],[]
for f in dataset_test:
  cand,txt=rank(path+f)
  lenght=0
  for c in cand:
    lenght+=len(cand[c])
  b=a+lenght
  pred,cf=rules(cand, prediction[a:b],coef[a:b])
  prediction_FINAL=prediction_FINAL+pred
  coef_FINAL=coef_FINAL+cf
  a=b

In [39]:
precision, recall, f1, _ = precision_recall_fscore_support(labels_test, prediction_FINAL, average='binary')
acc = accuracy_score(labels_test, prediction_FINAL)
print('accuracy: ',acc)
print('precision: ',precision)
print('recall: ',recall)
print('f1: ',f1)

accuracy:  0.8585707146426786
precision:  0.6601705237515225
recall:  0.6537997587454765
f1:  0.656969696969697


In [40]:
mAP(dataset_test,coef_FINAL,labels_test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


0.7681850697685044

In [41]:
mRank(dataset_test,prediction_FINAL,labels_test,coef_FINAL)

0.9366626065773447